In [1]:
from fetch import fetch
import ast

In [2]:
# convert source code into ast, a tree representation
with open('hello.py') as f: 
    source = f.read()

tree = ast.parse(source)
tree.body

In [3]:
# a flattened view of the ast - this is only for illustrative purposes and we wont use this
for statement in tree.body:
    print(ast.dump(statement), '\n')

Import(names=[alias(name='ast', asname=None)]) 

Import(names=[alias(name='numpy', asname='np')]) 

FunctionDef(name='a', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='z', ctx=Store())], value=Num(n=0)), Assign(targets=[Name(id='x', ctx=Store())], value=Call(func=Attribute(value=Attribute(value=Name(id='np', ctx=Load()), attr='random', ctx=Load()), attr='normal', ctx=Load()), args=[], keywords=[])), Return(value=BinOp(left=BinOp(left=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[]), op=Add(), right=Call(func=Name(id='b', ctx=Load()), args=[], keywords=[])), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[], returns=None) 

FunctionDef(name='b', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=2), op=Add(), right=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[])))], decorator_list=[], returns=None) 


# [example](https://suhas.org/function-call-ast-python/) - carve out call graph
starting with the assign statement `y = a()` we will extract a call graph from source.

we first find the ast node corresponding to `y`.

In [4]:
import ast
import astor  # https://stackoverflow.com/questions/36022935/rewriting-code-with-ast-python
import io
from contextlib import redirect_stdout


def name2idx(target, tree):
    """return index corresponding to a node's name in an ast tree.

    name2idx only looks for function definitions but we can modify it to look for ast.Import, 
    ast.Assign, etc.
    """
    for idx, node in enumerate(tree.body):        
        if isinstance(node, ast.FunctionDef) and (node.name == target):
            return idx
        elif isinstance(node, ast.Assign) and (node.targets[0].id == target):
            return idx
        else:
            pass
    return None


def fetch(start_name, tree):
    """
    start_name is a str, eg "a".
    
    tree is an ast tree object, usually constructed with `tree = ast.parse(source)`.
    
    # Python itself doesn’t provide a way to turn a compiled code object into an AST, or an AST into a string of code. https://greentreesnakes.readthedocs.io/en/latest/tofrom.html
    """
    stack = [start_name]
    seen = []
    while len(stack) > 0:
        name = stack.pop()        
        seen.append(name)  # no double counting

        # find index corresponding to name
        idx = name2idx(name, tree)  

        # now that we have the index, retrieve correspoding node from ast tree
        curr = tree.body[idx]
        
        # print function name
        if isinstance(curr, ast.FunctionDef):
            print(curr.name)

        # print its source code before moving on
        # print(astor.to_source(curr))

        # find all callables for current node
        callable_name = find_callables(curr)  

        # and add callable name to stack if we havent seen it yet
        for name in callable_name:
            print("adding", name)
            if (name not in seen) and (name not in stack):
                stack.append(name)
                

def find_callables(curr):
    """for a given curr, return a list of its callables (by name, not ast node). 
    """
    f = io.StringIO()  # https://stackoverflow.com/questions/16571150/how-to-capture-stdout-output-from-a-python-function-call
    with redirect_stdout(f):
        FunctionCallVisitor().visit(curr)  # only prints node attributes, does not return ast nodes
    out = f.getvalue()  # capture stdout and return it as a list
    return out.split()


class FunctionCallVisitor(ast.NodeVisitor):
    """print all children Call nodes, starting from curr.
    
    we will use this to find all callables ("Call" nodes in an ast). then, once we
    are on a Call node, we print its information. a Call node could be defined by 
    FunctionDef but also ImportFrom.
    
    if we wanted to visit other types of nodes in the ast, we would replace visit_Call
    with visit_Assign, visit_Import, etc...
    
    https://stackoverflow.com/questions/1515357/simple-example-of-how-to-use-ast-nodevisitor
    """
    def visit_Call(self, node):
        # print(ast.dump(node))
        print(node)

In [5]:
# recursively find all callables, starting from y=a()
# fetch("y", tree)

In [6]:
FunctionCallVisitor().visit(tree.body[2])

In [7]:
tree.body

In [8]:
tree.body[0].names[0]

In [9]:
tree.body[0].names[0].name

'ast'

In [10]:
# once i have the names, i could check the global environment

def find_calls(n):

    if isinstance(n, ast.Call):
        print("ast.call", n)
        func = n.func  # all calls have a func, args, keyword
        find_calls(func)
        
    elif isinstance(n, ast.alias):
        print("ast.alias", n.name, n.asname)  # alias or real name
    
    elif isinstance(n, ast.Import):
        # all ast.imports have a list of names
        print("ast.import", n)
        names = n.names
        for name in names:
            find_calls(name)
    
    elif isinstance(n, ast.FunctionDef):
        # all functiondefs have bodies
        bodies = n.body  # list of bodies
        for body in bodies:
            find_calls(body)
    
    elif isinstance(n, ast.Assign):
        # ast.assigns have targets and values
        print("ast.assign", n)
        value = n.value
        find_calls(value)
        
        targets = n.targets  # a list of targets
        for target in targets:
            find_calls(target)
    
    elif isinstance(n, ast.Return):
        # returns have values
        print("ast.return", n)
        value = n.value
        find_calls(value)
        
    elif isinstance(n, ast.BinOp):
        # all binops have left and right
        print("ast.binop", n)
        left = n.left
        right = n.right
        find_calls(left)
        find_calls(right)
        
    elif isinstance(n, ast.Name):
        print("ast.name", n.id)

    elif isinstance(n, ast.Attribute):
        print("ast.attribute", n.attr)
        # all attributes have a value
        value = n.value
        find_calls(value)
    
    elif isinstance(n, ast.Num):
        # all nums have a n
        print("ast.num", n.n)
    
    else:
        print(">>> hello", n.__class__)

In [21]:
find_calls(tree.body[2])

ast.assign <_ast.Assign object at 0x10f194d90>
ast.num 0
ast.name z
ast.assign <_ast.Assign object at 0x10f194e50>
ast.call <_ast.Call object at 0x10f194ed0>
ast.attribute normal
ast.attribute random
ast.name np
ast.name x
ast.return <_ast.Return object at 0x10f194fd0>
ast.binop <_ast.BinOp object at 0x10f19e050>
ast.binop <_ast.BinOp object at 0x10f19e090>
ast.call <_ast.Call object at 0x10f19e0d0>
ast.name c
ast.call <_ast.Call object at 0x10f19e150>
ast.name b
ast.name x


In [22]:
def is_defined(target):
    """determine if target is a user defined method.
    """
    for idx, node in enumerate(tree.body):
        if isinstance(node, ast.FunctionDef) and (node.name == target):
            return True
    return False

In [20]:
is_defined("x")

False

In [25]:
dir(tree.body[2])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'args',
 'body',
 'col_offset',
 'decorator_list',
 'lineno',
 'name',
 'returns']

In [26]:
tree.body[2]._fields


('name', 'args', 'body', 'decorator_list', 'returns')